In [1]:
from pathlib import Path
import optuna
import subprocess
import pickle
import numpy as np

In [2]:
def run_command(command):
    result = subprocess.run(command, capture_output=True)
    assert (
        result.returncode == 0
    ), f"""
command:
{' '.join(command)}
returncode: {result.returncode}
stderr:
{result.stderr.decode()}
stdout:
{result.stdout.decode()}
"""
    return result

In [3]:
def objective(trial):
    global acopp_dir, instance_name, debug_mode

    ants = trial.suggest_int("ants", 10, 500)
    alpha = trial.suggest_float("alpha", 0, 10)
    beta = trial.suggest_float("beta", 0, 10)
    rho = trial.suggest_float("rho", 0, 1)
    q0 = trial.suggest_float("q0", 0, 1)
    ptries = trial.suggest_int("ptries", 1, 5)

    cluster_size = trial.suggest_int("cluster_size", 10, 128)
    sector = trial.suggest_int("sector", 2, cluster_size)
    n_cluster = trial.suggest_int("n_cluster", 2, 1000 // cluster_size + 1)

    command = [
        'python3',
        f'{acopp_dir}/run.py',
        '--acopp_dir',
        str(acopp_dir),
        '--instance_name',
        instance_name,
        '--aaco_nc',
        '--run_only',
        '--experiment',
        '--no_log',
        # '--sol_dir',
        # '../../solutions/temp',
        '--silent',
        '1',
        '--random_seed',
        str(np.random.randint(1,2147483647)),
        '--alpha',
        str(alpha),
        '--beta',
        str(beta),
        '--rho',
        str(rho),
        '--q0',
        str(q0),
        '--ants',
        str(ants),
        '--ptries',
        str(ptries),
        '--sector',
        str(sector),
        '--cluster_size',
        str(cluster_size),
        '--n_cluster',
        str(n_cluster),
    ]
    if debug_mode:
        command += ["--time", "10"]
    
    try:
        result = run_command(command)
        stdout_log = result.stdout.decode()
        profit = int(stdout_log)
    except:
        profit = -1

    return profit

In [4]:
debug_mode = False
acopp_dir = Path("./")
instance_name = 'eil51_10_usw_10_03'
n_trials = 5000
n_jobs = 6
if debug_mode:
    n_trials = 2
    n_jobs = 2
save_path = Path("./study.pkl")
sampler = optuna.samplers.TPESampler()

In [5]:
command = [
    'python3',
    f'{acopp_dir}/run.py',
    '--acopp_dir',
    str(acopp_dir),
    '--build_only',
    '--experiment'
    ]
result = run_command(command)
print(result.stdout.decode())

$ cmake -DCMAKE_EXPORT_COMPILE_COMMANDS:BOOL=TRUE -G Unix Makefiles -S. -B./build -DCMAKE_BUILD_TYPE:STRING=Release
-- Configuring done
-- Generating done
-- Build files have been written to: /mnt/c/home/vincent/data/mega/Projects/Work/Public/acoplusplus_thop_modified/src/aco++/build



$ make -C ./build
make: Entering directory '/mnt/c/home/vincent/data/mega/Projects/Work/Public/acoplusplus_thop_modified/src/aco++/build'
make[1]: Entering directory '/mnt/c/home/vincent/data/mega/Projects/Work/Public/acoplusplus_thop_modified/src/aco++/build'
make[2]: Entering directory '/mnt/c/home/vincent/data/mega/Projects/Work/Public/acoplusplus_thop_modified/src/aco++/build'
Consolidate compiler generated dependencies of target acothop
make[2]: Leaving directory '/mnt/c/home/vincent/data/mega/Projects/Work/Public/acoplusplus_thop_modified/src/aco++/build'
make[2]: Entering directory '/mnt/c/home/vincent/data/mega/Projects/Work/Public/acoplusplus_thop_modified/src/aco++/build'
[  9%] Linking CXX ex

In [6]:
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)

[I 2023-06-01 19:38:31,588] A new study created in memory with name: no-name-b656d91c-f748-494d-a92d-7c7e3b87fd2d
[I 2023-06-01 19:38:41,944] Trial 1 finished with value: 212611.0 and parameters: {'ants': 182, 'alpha': 6.091843836028321, 'beta': 2.3447041673855953, 'rho': 0.5517627273126631, 'q0': 0.9214273557833694, 'ptries': 1, 'cluster_size': 104, 'sector': 9, 'n_cluster': 10}. Best is trial 0 with value: 214807.0.
[I 2023-06-01 19:38:41,946] Trial 0 finished with value: 214807.0 and parameters: {'ants': 465, 'alpha': 8.893845957942668, 'beta': 8.575727655467338, 'rho': 0.4715700371433933, 'q0': 0.6682694943139911, 'ptries': 5, 'cluster_size': 27, 'sector': 6, 'n_cluster': 3}. Best is trial 0 with value: 214807.0.


In [7]:
with open(save_path, "wb") as f:
    pickle.dump(study, f)